In [ ]:
using Pkg
Pkg.activate("/home/julisn/Codes/LadderDGA.jl/")
using LadderDGA
using Plots, LaTeXStrings

cfg = "/home/julisn/Hamburg/ED_data/conf_b7.toml"
wp, mP, sP, env, kGridsStr = readConfig(cfg)
lDGAhelper = setup_LDGA(kGridsStr[1], mP, sP, env)

  Activating project at `~/Codes/LadderDGA.jl`


[ Info: Reading Inputs...
┌ Error: ERROR: input file at location /home/julian/Hamburg/ED_data/b7.jld2 not found!
└ @ LadderDGA ~/Codes/LadderDGA.jl/src/IO.jl:64
